# NLP

In [13]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json

In [2]:
def connect_to_database(PATH1, attach=None):
    if os.path.isfile(PATH1):
        conn = sqlite3.connect(PATH1)
        cursor = conn.cursor()
    else:
        print("Database file does not exists")
        
    if attach:
        if os.path.isfile(attach):
            cursor.execute(f"ATTACH '{attach}' AS meta")
        else:
            print("Attach database file note found")
    
    return cursor

cursor = connect_to_database('mxm_dataset.db', 'track_metadata.db')

In [3]:
# How many songs in lyrics database?
# How many in track metadata?
# Create a view consisting of just the tracks and data in lyrics database
query = """SELECT
                COUNT(DISTINCT(track_id))
            FROM
                lyrics
        """
cursor.execute(query)
num_lyric_tracks = cursor.fetchall()
print(num_lyric_tracks)

query = """SELECT
                COUNT(DISTINCT(track_id)) 
           FROM 
                meta.songs
        """

cursor.execute(query)
num_tracks_total = cursor.fetchall()
print(num_tracks_total)

query = """CREATE TABLE if not exists
                lyric_metadata
            AS
            SELECT 
                * 
            FROM 
                meta.songs
            WHERE
                meta.songs.track_id
            IN
                (SELECT
                     DISTINCT(track_id)
                 FROM
                     lyrics)"""
cursor.execute(query)

[(237662,)]
[(1000000,)]


In [11]:
def link_track_to_lyrics(cursor, track_id):
    
    """For each track_id, fill the dictionary with each word and it's corresponding count"""
    query = """SELECT word, count FROM lyrics WHERE track_id = ?"""
    id = (track_id,)
    cursor.execute(query, id)
    word_counts = dict(cursor.fetchall())
    
    return word_counts

def extract_track_ids(cursor):
    """Extract all the distinct track_ids to iterate through and create bag of words"""
    
    query = """SELECT DISTINCT(track_id) FROM lyrics"""
    cursor.execute(query)
    track_ids = cursor.fetchall()
    track_lyric_dict = {id[0]:link_track_to_lyrics(cursor, id[0]) for id in track_ids}
    
    return track_lyric_dict
    
def map_words_to_tracks(cursor):
    
    """Bag of words for each track are in long format. This function takes
    these data and transforms them into 1 row for each song"""
    
    query = """SELECT word FROM words"""
    cursor.execute(query)
    words = cursor.fetchall()
    words = [word[0] for word in words]
    word_mappings = {i:word for i, word in enumerate(words)}
    
    track_lyric_dict = extract_track_ids(cursor)
    track_lyric_mappings = {i:track for i,track in enumerate(track_lyric_dict.keys())}
    n_rows = len(track_lyric_dict)
    n_cols = len(words)
    matrix = np.zeros([n_rows, n_cols])
    
    if not os.path.isfile("track_word_matrix.npy"):
        for track_index, track in track_lyric_mappings.items():
            for word_index, word in word_mappings.items():
                matrix[track_index, word_index] = track_lyric_dict[track].get(word, 0)
    else:
        matrix = np.load("track_word_matrix.npy")
        matrix = matrix1[:]
        del matrix1
            
    return word_mappings, track_lyric_mappings, matrix
    
    
word_mappings, track_lyric_mappings ,track_word_matrix = map_words_to_tracks(cursor)

In [15]:
with open('word_mappings', 'w') as f:
    json.dump(word_mappings, f)

In [16]:
with open('track_lyric_mapping', 'w') as f:
    json.dump(track_lyric_mappings, f)

In [6]:
np.save('track_word_matrix.npy', track_word_matrix)

In [12]:
# Free up memory
del word_mappings
del track_lyric_mappings
del tf_idf_matrix
del track_word_matrix

In [8]:
## Calucuating tf-idf - http://www.tfidf.com/

def tf_matrix(matrix):
    """Caluclate the relative word frequency for each word in a song"""
    tf_matrix = matrix / matrix.sum(axis=1).reshape(-1,1)
    
    return tf_matrix

def idf_matrix(matrix):
    """Calculate IDF"""
    total_tracks = matrix.shape[0]
    track_with_words = (matrix > 0).sum(axis=0).reshape(1,-1)
    idf = np.log(total_tracks / track_with_words)
    
    return idf

def tf_idf(matrix):
    """Calculate tf-idf"""
    tf = tf_matrix(matrix)
    idf = idf_matrix(matrix)
    return tf * idf    
    

tf_idf_matrix = tf_idf(track_word_matrix)

In [9]:
# Save arrays for use in Dash app
np.save('tf_idf_matrix.npy', tf_idf_matrix)

In [7]:
## cosine similarity 
# dot product(a,b) /length(a)*length(b)
# Given a song, find 10 most similar

# Create track_id mapper
def calc_10_most_similar(song_id, tf_idf_matrix, n=10):
    """Returns the index of the 10 most similar songs as well as a cosine similarity score"""
    numerator = np.dot(tf_idf_matrix, tf_idf_matrix[song_id,:].reshape(-1,1))
    denominator = np.linalg.norm(tf_idf_matrix[song_id]) * np.linalg.norm(tf_idf_matrix, axis=1).reshape(-1,1)
    cosine_sim = (numerator / denominator).reshape(-1)
    sort_indices = np.argsort(-cosine_sim)
    return sort_indices[:n], cosine_sim

In [8]:
def extract_most_similar_names(song_id, tf_idf_matrix):
    """From a given song ID, returns the top n songs (artist and track) as well as cosine similarity"""
    sort_indices, cosine_sim = calc_10_most_similar(song_id, tf_idf_matrix)
    
    # Return the track_ids of the top n most similar along with cosine similarity
    top_n = [track_lyric_mappings[x] for x in sort_indices]
    cosine_sim = [x for x in cosine_sim[sort_indices]]
    
    songs = []
    for idx, song in enumerate(top_n):
        query_str = (song,)
        cursor.execute("SELECT title, artist_name FROM lyric_metadata WHERE track_id = ?", query_str)
        song = cursor.fetchall()
        songs.append((*song,cosine_sim[idx]))
                     
    return songs


In [34]:
def track_id_to_index(track_id, track_lyric_mappings):
    for key, value in track_lyric_mappings.items():
        if value == track_id:
            return int(key)
        
track_id_to_index("TRGIATU128F427878D", tl)

58041

In [138]:
extract_most_similar_names(58041, tf_idf_matrix)

[(('The Rising', 'Bruce Springsteen'), 1.0),
 (('Prickly Thorn_ But Sweetly Worn (Live)', 'The White Stripes'),
  0.8718244730848413),
 (('Toe Hold (LP Version)', 'Wilson Pickett'), 0.7876098977053478),
 (('We Live', 'Bosson'), 0.7832116066128567),
 (('Ilham', 'Souad Massi'), 0.7189643953697207),
 (('Denia', 'Manu Chao'), 0.6570293712624922),
 (('Hagda Wala Akter', 'Souad Massi'), 0.6217219904757953),
 (('The Last Polka', 'Ben Folds Five'), 0.5797917438223231),
 (('The Last Polka', 'Ben Folds'), 0.5751670401107143),
 (('Omri', 'Faudel'), 0.5231962553654297)]

In [5]:
query = """SELECT artist_name, title FROM lyric_metadata"""
cursor.execute(query)
artist_song_list = cursor.fetchall()
artist_list = set([x[0] for x in artist_song_list])

## Using np.memmap to load the numpy arrays in blocks that fit into memory

In [4]:
n_rows = 237662
n_cols = 5000

data = np.load('tf_idf_matrix.npy', mmap_mode='r')